In [1]:
import os
import re
import csv
import time 
import requests
from bs4 import BeautifulSoup
import datetime as dt
from base64 import b64encode
from pickle import load, dump

In [2]:
date_info = dt.date.today().isocalendar()

year = date_info[0]
week_num = date_info[1]

In [3]:
url_for_scrape = 'https://www.metacritic.com/browse/albums/release-date/new-releases/date'
user_agent = {'User-agent': 'Mozilla/5.0'}
# send response
response_score = requests.get(url_for_scrape, headers = user_agent)
# scrape website into variable to parse
soup_score = BeautifulSoup(response_score.text, 'html.parser')
# create list for dictionarys 
album_dicts = []
# create soup 
print(soup_score)

<!DOCTYPE html>

<html xmlns:fb="http://ogp.me/ns/fb#" xmlns:og="http://opengraphprotocol.org/schema/">
<head>
<title>New Album Releases by Date - Metacritic</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="See how well critics are rating new Music and Albums at Metacritic" name="description"/>
<meta content="Metacritic" name="application-name"/>
<meta content="#000000" name="msapplication-TileColor"/>
<meta content="/images/win8tile/76bf1426-2886-4b87-ae1c-06424b6bb8a2.png" name="msapplication-TileImage"/>
<meta content="New Album Releases by Date" property="og:title"/>
<meta content="website" property="og:type"/>
<meta content="https://www.metacritic.com/browse/albums/release-date/new-releases/date" property="og:url"/>
<meta content="https://static.metacritic.com/images/icons/mc_fb_og.png" property="og:image"/>
<meta content="Metacritic" property="og:site_name"/>
<meta content="123113677890173" property="fb:app_id"/>
<meta content="@metacrit

In [4]:
for item in soup_score.find_all('td', class_='clamp-summary-wrap'):
    album_dict = {}
    ###        
    # scrape date
    #sqlite doesn't support datetime objects date column will be string
    #make an objec of date to extract week num and year
    ###
    date_string = (item.find('div', class_='clamp-details').find('span').text)
    date_obj = dt.datetime.strptime(date_string, '%B %d, %Y')
    album_year, album_week_num, day_of_week = date_obj.isocalendar()
    ###
    # limit scrape to previous three weeks of albums and current 
    # year of release date
    ###
    if album_week_num < (week_num - 7) or album_year < year:
        break   
    else:
        album_dict['date']=date_string
        album_dict['year'] = year
        album_dict['week_num']=album_week_num
        ###
        # scrape album name
        # regular expressions to prepare album and artist names for 
        # search in spotify
        ####
        album_raw = item.find('a', class_= 'title').text
        album_dict['album_raw'] = album_raw
        al = re.sub(r'[^-A-Za-z0-9!áéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ ]+', '', album_raw)
        album_clean = re.sub(' +', ' ', al)
        album_dict['album']= album_clean
        ###
        # scrape artist name and strip white space and extra characters
        ###
        artist_raw = item.find('div', class_='artist').text.strip().lstrip('by ')
        album_dict['artist_raw'] = artist_raw
        ar = re.sub(r'[^-A-Za-z0-9!áéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ ]+', '', artist_raw)
        artist_clean= re.sub(' +', ' ', ar)
        album_dict['artist']=artist_clean

        ###
        # Handle for varaitions in classes for critic name by pattern matching with regular expression.
        # then scrape critic and user scores
        ###
        meta_critic_pattern = re.compile('^metascore_w large')
        meta_user_pattern = re.compile('^metascore_w user')
        album_dict['meta_score']=int(item.find('div', class_= meta_critic_pattern).text)
        user_string = (item.find('div', class_= meta_user_pattern).text)
        ###
        # Handle for variations in classes for user by filtering out strings from scores to and casting to ints
        ###
        if user_string == 'tbd':
            album_dict['user_score'] = 0
        else:
            user_score = int(float(user_string)*10)
            album_dict['user_score']=user_score
        album_dicts.append(album_dict)
album_dicts

[{'date': 'March 4, 2022',
  'year': 2022,
  'week_num': 9,
  'album_raw': 'Things Are Great',
  'album': 'Things Are Great',
  'artist_raw': 'Band of Horses',
  'artist': 'Band of Horses',
  'meta_score': 83,
  'user_score': 0},
 {'date': 'February 25, 2022',
  'year': 2022,
  'week_num': 8,
  'album_raw': 'The Tipping Point',
  'album': 'The Tipping Point',
  'artist_raw': 'Tears for Fears',
  'artist': 'Tears for Fears',
  'meta_score': 87,
  'user_score': 0},
 {'date': 'February 18, 2022',
  'year': 2022,
  'week_num': 7,
  'album_raw': 'Once Twice Melody',
  'album': 'Once Twice Melody',
  'artist_raw': 'Beach House',
  'artist': 'Beach House',
  'meta_score': 85,
  'user_score': 95},
 {'date': 'February 18, 2022',
  'year': 2022,
  'week_num': 7,
  'album_raw': 'LIFE ON EARTH',
  'album': 'LIFE ON EARTH',
  'artist_raw': 'Hurray for the Riff Raff',
  'artist': 'Hurray for the Riff Raff',
  'meta_score': 86,
  'user_score': 88},
 {'date': 'February 18, 2022',
  'year': 2022,
  'we

In [8]:
for album in album_dicts:
    if album['artist_raw']!=album['artist']:
        print(f"{album['artist_raw']} / {album['artist']}")

Shovels & Rope / Shovels Rope
Mary J. Blige / Mary J Blige
Author & Punisher / Author Punisher
The Reds, Pinks & Purples / The Reds Pinks Purples
Black Country, New Road / Black Country New Road
Anaïs Mitchell / Anas Mitchell
St. Paul & the Broken Bones / St Paul the Broken Bones
MØ / M
Years & Years / Years Years
Keb' Mo' / Keb Mo
Aoife O'Donovan / Aoife ODonovan
Elvis Costello & the Imposters / Elvis Costello the Imposters


In [6]:
def scrape_reviews(album_dicts, week_num, user_agent, al, ar):
    url_beginning ='https://www.metacritic.com/music/'
    genre_lsts = []
    for album_dict in album_dicts:
        # only check for the last 3 weeks of albums
        if album_dict['week_num'] < (week_num - 6):
            break
        else:
            
            url_end = f"{album_dict['album']}/{album_dict['artist']}".replace(" ", "-").lower() 
            url = url_beginning + url_end
            ###
            # creating pickles to prevent disconnect from too many pings
            # check if a pickle file exists for the current day 
            # if not run scrape for url and create a pickle file
            ###
            picklefile = os.path.join('..', 'data', 'pickles',str(b64encode(url.encode('utf-8')),'utf-8'))
            
            if os.path.exists(picklefile) and dt.fromtimestamp(os.path.getctime(picklefile)).day==dt.now().date().day:
                with open(picklefile,'rb') as pickleload:
                    content = load(pickleload)
            else:
                content = requests.get(url, headers = user_agent)
                with open(picklefile,'wb') as picklesave:
                    dump(content, picklesave)
            # sleep so as to not get bounced from connection
                    time.sleep(3)
            # scrape website into variable to parse
            soup_reviews = BeautifulSoup(content.text, 'html.parser')
            # scrape num of critical reviews
            try:
                num_rev=(soup_reviews.find('span', itemprop="reviewCount"))
                album_dict['crit_num'] = num_rev.text.strip()
            except:
                album_dict['crit_num'] = None
            # scrape num of user reviews
            if album_dict['user_score'] == 0:
                album_dict['user_num'] = 0
            else:
                try:
                    album_dict['user_num'] = int(soup_reviews.find_all("a", href=f"/music/{url_end}/user-reviews")[2].text.split()[0])
                except:
                    album_dict['user_num'] = 0
            # scrape record labels
            try:
                label_class = soup_reviews.find_all("span", itemprop="name")
                album_dict['label'] = label_class[2].text.strip()
            except:
                album_dict['label'] = None
            ###
            # scrape genre and ensure pop/rock is not provided in case of 
            # alternative genres being present. Too many artist were being
            # represented as pop/rock
            ###
            try:
                genres_lst = []
                genre_elements = soup_reviews.find_all("span", itemprop="genre")
                for genre in genre_elements:
                    genres_lst.append(genre.text)
            except:
                album_dict['genre'] = 'unknown'
            if len(genres_lst) == 0:
                album_dict['genre'] = 'unknown'
            elif len(genres_lst) == 1:
                album_dict['genre'] = genres_lst[0]
            else:
                for g in genres_lst:
                    if g != 'Pop/Rock':
                        album_dict['genre'] = g
            #             break

            # genre_lsts.append(f"{album_dict['genre']}  for artist: {album_dict['artist']}")
            # genre_lsts.append(f'out of: {genres_lst}')
     
    # with  open(text_file_path, "r") as file:
    #     for f in file:
    #         print(f.read())

#open and read the file after the appending:

    write_csv(album_dicts, week_num)
        

In [7]:
def write_csv(album_dicts, week_num):
    # write dictionary to csv
    # csv variables
    output_path = os.path.join('..', 'data', 'meta_scrape.csv')
    # create variable for data to be written
    keys = album_dicts[0].keys()
    # output to csv
    print('writing')
    with open(output_path, 'w', encoding='utf-8', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, keys)
        writer.writeheader()
        writer.writerows(album_dicts)
    # call create playlist script
     